#### Code source: 'https://medium.com/@lope.ai/recommendation-systems-from-scratch-in-python-pytholabs-6946491e76c2'

### Load movielens data

In [1]:
import pandas as pd
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

# pass in column names for each CSV as the column name is not given in the file and read them using pandas.
# You can check the column names from the readme file
#Reading users file:
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('ml-100k/u.user', sep='|', names=u_cols,encoding='latin-1')

#Reading ratings file:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv('ml-100k/u.data', sep='\t', names=r_cols,encoding='latin-1')

#Reading items file:
i_cols = ['movie id', 'movie title' ,'release date','video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure',
'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

items = pd.read_csv('ml-100k/u.item', sep='|', names=i_cols,encoding='latin-1')

In C:\Users\Arunabh\miniconda3\envs\tensorflow\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\Arunabh\miniconda3\envs\tensorflow\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\Arunabh\miniconda3\envs\tensorflow\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In C:\Users\Arunabh\miniconda3\envs\tensorflow\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\Arun

In [3]:
print(users.shape)
users.head()

(943, 5)


,user_id,age,sex,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [4]:
print(ratings.shape)
ratings.head()

(100000, 4)


,user_id,movie_id,rating,unix_timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [5]:
print(items.shape)
items.head()

(1682, 24)


,movie id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [6]:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']

ratings_train = pd.read_csv('ml-100k/ua.base', sep='\t', names=r_cols, encoding='latin-1')

ratings_test = pd.read_csv('ml-100k/ua.test', sep='\t', names=r_cols, encoding='latin-1')

ratings_train.shape, ratings_test.shape

((90570, 4), (9430, 4))

### Use Collaborative Filtering

In [7]:
n_users = ratings.user_id.unique().shape[0]
n_items = ratings.movie_id.unique().shape[0]

In [8]:
data_matrix = np.zeros((n_users, n_items))

In [9]:
data_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [10]:
for line in ratings.itertuples():
    data_matrix[line[1]-1, line[2]-1] = line[3]

* line[1] is the userId and we are subtracting 1 from it since array indexing starts from 0 = row

* line[2]-1 is the movie id = column

* now at that specific row and column i.e, user and movie we will add line[3] which is the movie rating
Now, when we have rating of all the movies given by each user in a matrix we will calculate the similarity. We can use the pairwise_distance function from sklearn to calculate the cosine similarity.

In [11]:
data_matrix

array([[5., 3., 4., ..., 0., 0., 0.],
       [4., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [5., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 5., 0., ..., 0., 0., 0.]])

In [12]:
data_matrix.shape

(943, 1682)

In [13]:
from sklearn.metrics.pairwise import pairwise_distances 
user_similarity = pairwise_distances(data_matrix, metric='cosine')
item_similarity = pairwise_distances(data_matrix.T, metric='cosine')

In [14]:
user_similarity

array([[0.        , 0.83306902, 0.95254046, ..., 0.85138306, 0.82049212,
        0.60182526],
       [0.83306902, 0.        , 0.88940868, ..., 0.83851522, 0.82773219,
        0.89420212],
       [0.95254046, 0.88940868, 0.        , ..., 0.89875744, 0.86658385,
        0.97344413],
       ...,
       [0.85138306, 0.83851522, 0.89875744, ..., 0.        , 0.8983582 ,
        0.90488042],
       [0.82049212, 0.82773219, 0.86658385, ..., 0.8983582 , 0.        ,
        0.81753534],
       [0.60182526, 0.89420212, 0.97344413, ..., 0.90488042, 0.81753534,
        0.        ]])

In [15]:
user_similarity[0,:].mean()

0.7540625611774527

In [18]:
user_similarity

array([[0.        , 0.83306902, 0.95254046, ..., 0.85138306, 0.82049212,
        0.60182526],
       [0.83306902, 0.        , 0.88940868, ..., 0.83851522, 0.82773219,
        0.89420212],
       [0.95254046, 0.88940868, 0.        , ..., 0.89875744, 0.86658385,
        0.97344413],
       ...,
       [0.85138306, 0.83851522, 0.89875744, ..., 0.        , 0.8983582 ,
        0.90488042],
       [0.82049212, 0.82773219, 0.86658385, ..., 0.8983582 , 0.        ,
        0.81753534],
       [0.60182526, 0.89420212, 0.97344413, ..., 0.90488042, 0.81753534,
        0.        ]])

In [17]:
user_similarity.mean(axis=1).reshape(-1,1)

array([[0.75406256],
       [0.80601624],
       [0.87587914],
       [0.87740429],
       [0.82679226],
       [0.76009093],
       [0.75652677],
       [0.82416084],
       [0.89603166],
       [0.78288379],
       [0.80552321],
       [0.83518952],
       [0.73146715],
       [0.79452057],
       [0.81436782],
       [0.77483133],
       [0.84305373],
       [0.76919441],
       [0.8895856 ],
       [0.83841579],
       [0.83518149],
       [0.81697161],
       [0.78190658],
       [0.79923573],
       [0.80888189],
       [0.75496001],
       [0.88105441],
       [0.8126099 ],
       [0.86727431],
       [0.83775238],
       [0.92024871],
       [0.82311181],
       [0.87119344],
       [0.92968465],
       [0.90199929],
       [0.9325982 ],
       [0.84103247],
       [0.8622213 ],
       [0.86062495],
       [0.8701152 ],
       [0.81804611],
       [0.77965805],
       [0.7305182 ],
       [0.75531179],
       [0.84319052],
       [0.83717281],
       [0.87797201],
       [0.833

In [29]:
np.abs(user_similarity).sum(axis=1)

array([711.08099519, 760.07331717, 825.95402725, 827.39224279,
       779.66510396, 716.76574749, 713.40474397, 777.18367345,
       844.95785479, 738.25941741, 759.6083914 , 787.5837141 ,
       689.77352349, 749.23289325, 767.9488538 , 730.66594402,
       794.99966418, 725.35033113, 838.87922505, 790.62608759,
       787.57614319, 770.40422666, 737.33790467, 753.67929565,
       762.77562448, 711.92729357, 830.83431315, 766.29113247,
       817.83967763, 790.00049674, 867.79453678, 776.19443727,
       821.53541302, 876.69262601, 850.58533255, 879.44009825,
       793.09362316, 813.0746822 , 811.56933214, 820.51863065,
       771.41748237, 735.21753652, 688.87865937, 712.25901375,
       795.12865583, 789.45396034, 827.92760469, 786.45655402,
       781.28197329, 858.78586253, 828.78338027, 786.99216242,
       792.56008095, 746.92036575, 809.75085538, 725.44741742,
       740.58904277, 731.54581269, 696.33734916, 743.26197507,
       824.87190201, 701.06611664, 758.99343356, 693.76

In [30]:
user_similarity[0,:].sum()

711.080995190338

In [28]:
np.array([np.abs(user_similarity).sum(axis=1)])

array([[711.08099519, 760.07331717, 825.95402725, 827.39224279,
        779.66510396, 716.76574749, 713.40474397, 777.18367345,
        844.95785479, 738.25941741, 759.6083914 , 787.5837141 ,
        689.77352349, 749.23289325, 767.9488538 , 730.66594402,
        794.99966418, 725.35033113, 838.87922505, 790.62608759,
        787.57614319, 770.40422666, 737.33790467, 753.67929565,
        762.77562448, 711.92729357, 830.83431315, 766.29113247,
        817.83967763, 790.00049674, 867.79453678, 776.19443727,
        821.53541302, 876.69262601, 850.58533255, 879.44009825,
        793.09362316, 813.0746822 , 811.56933214, 820.51863065,
        771.41748237, 735.21753652, 688.87865937, 712.25901375,
        795.12865583, 789.45396034, 827.92760469, 786.45655402,
        781.28197329, 858.78586253, 828.78338027, 786.99216242,
        792.56008095, 746.92036575, 809.75085538, 725.44741742,
        740.58904277, 731.54581269, 696.33734916, 743.26197507,
        824.87190201, 701.06611664, 758.

This gives us the item-item and user-user similarity in an array form. The next step is to make predictions based on these similarities. Let’s define a function to do just that.

In [19]:

def predict(ratings, similarity, type='user'):
    
    if type == 'user':
        
        pred = similarity.dot(ratings)/ np.array([np.abs(similarity).sum(axis=1)]).T
    
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])
    
    return pred

In [20]:
user_prediction = predict(data_matrix, user_similarity, type='user')
item_prediction = predict(data_matrix, item_similarity, type='item')

In [23]:
user_prediction

array([[1.67437294e+00, 3.43349631e-01, 2.38970693e-01, ...,
        2.63729634e-03, 2.09562375e-03, 1.81808094e-03],
       [1.85328696e+00, 4.74238792e-01, 2.86377496e-01, ...,
        1.82071483e-03, 3.28031113e-03, 3.48678052e-03],
       [1.93413349e+00, 4.67276838e-01, 2.97058358e-01, ...,
        1.23728002e-03, 3.26099444e-03, 3.46463391e-03],
       ...,
       [1.76113528e+00, 4.44889039e-01, 2.71815487e-01, ...,
        2.26853225e-03, 3.04563616e-03, 3.20638348e-03],
       [1.82156495e+00, 4.16001047e-01, 2.86652414e-01, ...,
        2.12869738e-03, 2.73640943e-03, 3.15369564e-03],
       [1.69421944e+00, 3.35436519e-01, 2.40751065e-01, ...,
        2.65489953e-03, 2.08622458e-03, 2.20269348e-03]])

In [24]:
item_prediction

(943, 1682)

In [25]:
item_prediction

array([[0.44627765, 0.475473  , 0.50593755, ..., 0.58815455, 0.5731069 ,
        0.56669645],
       [0.10854432, 0.13295661, 0.12558851, ..., 0.13445801, 0.13657587,
        0.13711081],
       [0.08568497, 0.09169006, 0.08764343, ..., 0.08465892, 0.08976784,
        0.09084451],
       ...,
       [0.03230047, 0.0450241 , 0.04292449, ..., 0.05302764, 0.0519099 ,
        0.05228033],
       [0.15777917, 0.17409459, 0.18900003, ..., 0.19979296, 0.19739388,
        0.20003117],
       [0.24767207, 0.24489212, 0.28263031, ..., 0.34410424, 0.33051406,
        0.33102478]])

In [48]:
ind=1
no_of_rec=5
np.argsort(user_prediction[ind,:])[-no_of_rec:]

array([257, 173, 180,  99,  49], dtype=int64)

In [40]:
user_similarity.shape

(943, 943)

In [42]:
data_matrix.shape

(943, 1682)

In [59]:
np.where(data_matrix[257,:]==5)[0][0:3]

[271, 285, 299]

In [28]:
movie_id_title_dict = dict(zip(items['movie id'], items['movie title']))


In [75]:
movie_id_title_dict[5]

'Copycat (1995)'

In [30]:
def recommend_movies(movie_id_title_dict = {},data=None,type='user', index_no=1, no_of_rec=5):
    if type == 'user':
        user_similarity = pairwise_distances(data, metric='cosine')
        user_prediction = predict(data,user_similarity,type='user')
        similar_user_list = list(np.argsort(user_prediction[index_no,:])[-no_of_rec:])
        recommended_movie_list= []
        movie_indices_list = []
        for user_no in similar_user_list:
            movie_indices = list(np.where(data[user_no,:]==5)[0][0:3])
            movie_indices_list.extend(movie_indices)
        
        for movie_index in movie_indices_list:
            recommended_movie_list.append(movie_id_title_dict[movie_index])
        return recommended_movie_list
    if type == 'item':
        item_similarity = pairwise_distances(data.T,metric='cosine')
        item_prediction = predict(data,item_similarity,type='item')
        similar_item_list = list(np.argsort(item_prediction[index_no,:])[-no_of_rec:])
        recommended_movie_list=[]
        for movie_index in similar_item_list:
            recommended_movie_list.append(movie_id_title_dict[movie_index])
        return recommended_movie_list

In [32]:
recommend_movies(movie_id_title_dict=movie_id_title_dict,data=data_matrix,index_no=80)

['Four Rooms (1995)',
 'Richard III (1995)',
 'Desperado (1995)',
 'Starship Troopers (1997)',
 'Secrets & Lies (1996)',
 'Hoodlum (1997)',
 'Rumble in the Bronx (1995)',
 'Citizen Ruth (1996)',
 'Midnight in the Garden of Good and Evil (1997)',
 '3 Ninjas: High Noon At Mega Mountain (1998)',
 'Apt Pupil (1998)',
 'Lost World: Jurassic Park, The (1997)',
 "Schindler's List (1993)",
 "Dante's Peak (1997)"]

In [41]:
recommend_movies(movie_id_title_dict=movie_id_title_dict,data=data_matrix,type='item',index_no=6)

['Innocent Sleep, The (1995)',
 'Second Jungle Book: Mowgli & Baloo, The (1997)',
 'Tetsuo II: Body Hammer (1992)',
 'Angel Baby (1995)',
 'Broken English (1996)']

In [40]:
items[items.index==6]

,movie id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
6,7,Twelve Monkeys (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Twelve%20Monk...,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [39]:
items.head(20)

,movie id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
5,6,Shanghai Triad (Yao a yao yao dao waipo qiao) ...,01-Jan-1995,NaN,http://us.imdb.com/Title?Yao+a+yao+yao+dao+wai...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,7,Twelve Monkeys (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Twelve%20Monk...,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
7,8,Babe (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Babe%20(1995),0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
8,9,Dead Man Walking (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Dead%20Man%20...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,10,Richard III (1995),22-Jan-1996,NaN,http://us.imdb.com/M/title-exact?Richard%20III...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [120]:
items

,movie id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677,1678,Mat' i syn (1997),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?Mat%27+i+syn+...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1678,1679,B. Monkey (1998),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?B%2E+Monkey+(...,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
1679,1680,Sliding Doors (1998),01-Jan-1998,NaN,http://us.imdb.com/Title?Sliding+Doors+(1998),0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1680,1681,You So Crazy (1994),01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?You%20So%20Cr...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
predict(data_matrix, item_similarity, type='item')

In [122]:
item_similarity.shape

(1682, 1682)